In [1]:
import numpy as np
import xarray as xr
import scipy.io as sio
import datetime as dt
import sys

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim


from torch.utils.data import Dataset, DataLoader
from skimage.transform import rescale as skrescale
from scipy import signal as ssignal

In [2]:
#vstring = int(sys.argv[1])
rseed = int(42)
#humidvar = sys.argv[3]
import random
random.seed(rseed)
np.random.seed(rseed)
torch.manual_seed(rseed)
torch.cuda.manual_seed(rseed)
torch.cuda.manual_seed_all(rseed)

In [3]:
opt_model = 'R18'

In [31]:
import torch.nn as nn
from vit_pytorch import ViT

class TransformerEncoder(nn.Module):
    def __init__(self, image_size, patch_size, num_layers, num_heads, hidden_dim, channels):
        super(TransformerEncoder, self).__init__()
        self.encoder = ViT(
            image_size=image_size,
            patch_size=patch_size,
            num_classes=hidden_dim,  # Output dimension of the ViT
            dim=hidden_dim,
            depth=num_layers,
            heads=num_heads,
            mlp_dim=hidden_dim * 4,
            pool='cls',
            channels=channels
        )
    
    def forward(self, x):
        z = self.encoder(x.type(torch.FloatTensor)).type(torch.FloatTensor)
        del x
        return z

In [6]:
def collect_norm_data_by_var(my_var):

    sfile_max = datadir + 'agg_40year/minmax/pt.max.%s.nc' % my_var
    sfile_min = datadir + 'agg_40year/minmax/pt.min.%s.nc' % my_var

    with xr.open_dataset(sfile_max) as inds:
        ds_Vmax = inds[my_var].values

    with xr.open_dataset(sfile_min) as inds:
        ds_Vmin = inds[my_var].values
        
    print(ds_Vmax, ds_Vmin)

    my_vmax = np.maximum(ds_Vmax, -1*ds_Vmin)
    Vmax, Vmin = my_vmax, -1*my_vmax
    
    infile = datadir + 'ERA5.SCL.850mb_anomaly.%s.1981-2020.daymean.nc' % (my_var)
    print('loading data from %s ...' % infile)
    with xr.open_dataset(infile) as inds:
        outdata = (inds[my_var].values[:,0,:,:]-Vmin)/(Vmax-Vmin)

    return outdata

In [7]:
mean5kernel = np.ones((5,5))/25

class TrainDataset(Dataset):
    '''
    Since we need to mannually normalize the data, let's create datasets elsewhere, and just aggreagate them here.
    Requires: T_full, H_full, W_full, U_full, V_full, Z_full
    '''

    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.sample_data = root_dir + 'ERA5.SCL.850mb_anomaly.W.1981-2020.daymean.nc'

    def __len__(self):
        with xr.open_dataset(self.sample_data) as inds:
            nt = inds['W'].shape[0]
        return int(nt) - 10

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # find a corresponding idx_pair, outside the 360-length window of idx
        # idx_pair = xxxx

        idx_pair = idx + 10


        sample_raw = np.zeros((6,102,202))
        sample_raw_pair = np.zeros((6,102,202))

        for i,fullds in zip(np.arange(6), [H_full, T_full, W_full, U_full, V_full, Z_full]):
            
            # construct input for idx
            # rescaling
            data_step1 = skrescale(fullds[idx], (2.5, 2.5), anti_aliasing=True)
            # mean using 5x5
            data_step2 = ssignal.convolve2d(data_step1, mean5kernel, boundary='symm', mode='same')
            sample_raw[i] = data_step2


            # construct input for idx_pair
            # rescaling
            data_step1 = skrescale(fullds[idx_pair], (2.5, 2.5), anti_aliasing=True)
            # mean using 5x5
            data_step2 = ssignal.convolve2d(data_step1, mean5kernel, boundary='symm', mode='same')
            sample_raw_pair[i] = data_step2

        return sample_raw, sample_raw_pair, idx

In [8]:
def SCLloss(my_x, my_y, my_temperature=0.5):
    '''
    my_x and my_y has a one-to-one pair. So there are in total N*N pairs. In these N*N, the diagonal pairs are positive,
     and the rest are negative. So we want to maximum diagonal while suppressing the rest.
    '''
    ns = my_x.shape[0]
    # use broadcasting to achieve pairwise cos. Note my_y.t() operation and dimension handling
    cos_matrix = torch.nn.functional.cosine_similarity(my_x[:,:,None], my_y.t()[None,:,:])/my_temperature
    similarity_matrix = torch.exp(cos_matrix)


    loss = torch.tensor([0.0], requires_grad=True)
    for i in np.arange(ns):
        loss = loss -1*torch.log(similarity_matrix[i,i]/(torch.sum(similarity_matrix[i,:])-similarity_matrix[i,i]))
        loss = loss -1*torch.log(similarity_matrix[i,i]/(torch.sum(similarity_matrix[:,i])-similarity_matrix[i,i]))

    loss = loss/(2*ns)

    return loss

In [8]:
rootdir = '/global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/'

datadir = '/global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/'

In [32]:
# 0. major parameters
if opt_model=='R18':
    batch_size = 128
elif opt_model=='R15':
    batch_size = 64
    
# 1. construct functions
if opt_model=='R18':
    encoder = torchvision.models.resnet18(weights=None)
elif opt_model=='R50':
    encoder = torchvision.models.resnet50(weights=None)

n_features = encoder.fc.in_features  # get dimensions of fc layer

# 2. construct two models, one with random parameters, one with pre-trained parameters
projection_dim = 256
SCL = SCL(True, encoder, projection_dim, n_features)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SCL = SCL.to(device)

In [33]:
# 3. load data

H_full = collect_norm_data_by_var('RH')
T_full = collect_norm_data_by_var('T')
U_full = collect_norm_data_by_var('U')
V_full = collect_norm_data_by_var('V')
W_full = collect_norm_data_by_var('W')
Z_full = collect_norm_data_by_var('Z')

print(Z_full.shape)

67.06845 -76.24047
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.RH.1981-2020.daymean.nc ...
20.074585 -34.17395
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.T.1981-2020.daymean.nc ...
26.455658 -35.592907
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.U.1981-2020.daymean.nc ...
44.648796 -33.113792
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.V.1981-2020.daymean.nc ...
7.01952 -5.8575945
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.W.1981-2020.daymean.nc ...
3010.4414 -4328.3145
loading data from /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/xdchen/anomaly/ERA5.SCL.850mb_anomaly.Z.1981-2020.daymean.nc ...
(14610, 41, 81)


In [34]:
train_dataset = TrainDataset(root_dir=datadir)

# turn off shuffle, so data is processed in the time order
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [35]:
print(torch.cuda.is_available())


False


In [36]:
import torch
import torch.nn.functional as F

class NT_XentLoss(torch.nn.Module):
    def __init__(self, batch_size, temperature, device):
        super(NT_XentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.device = device
        self.mask = self._get_mask(batch_size)
        self.criterion = torch.nn.CrossEntropyLoss(reduction="sum")

    def _get_mask(self, batch_size):
        N = 2 * batch_size
        mask = torch.ones((N, N), dtype=bool)
        mask = mask.fill_diagonal_(0)
        for i in range(batch_size):
            mask[i, batch_size + i] = 0
            mask[batch_size + i, i] = 0
        return mask

    def forward(self, zis, zjs):
        N = 2 * self.batch_size
        zis = F.normalize(zis, dim=1)
        zjs = F.normalize(zjs, dim=1)
        representations = torch.cat([zis, zjs], dim=0)

        similarity_matrix = torch.matmul(representations, representations.T)
        similarity_matrix = similarity_matrix / self.temperature

        positives = torch.cat([torch.diag(similarity_matrix, self.batch_size),
                               torch.diag(similarity_matrix, -self.batch_size)], dim=0)
        negatives = similarity_matrix[self.mask].view(N, -1)

        labels = torch.zeros(N).to(self.device).long()
        logits = torch.cat([positives.unsqueeze(1), negatives], dim=1)

        loss = self.criterion(logits, labels)
        loss = loss / N
        return loss

In [37]:
outdata = np.zeros((2, H_full.shape[0], n_features))
outindex = np.zeros(H_full.shape[0])

sindex = -1*batch_size
eindex = 0

criterion = NT_XentLoss(batch_size=batch_size, temperature=0.5, device='cpu')
optimizer = optim.Adam(SCL.parameters(), lr=0.001)

print(dt.datetime.now())
for step, data in enumerate(train_dataloader):
    print(f'step: {step+1}')
    optimizer.zero_grad()
    testout = SCL(data[0].to(device), data[1].to(device))
    print(testout[0].shape)
    print(testout[1].shape)
    loss = criterion(testout[0], testout[1])
    loss.backward()
    optimizer.step()
    print(f"Step [{step+1}/10], Loss: {loss.item():.4f}")

    print(len(testout))
    print(testout[0].shape)


    sindex = eindex
    eindex += testout[0].shape[0]

    outdata[0,sindex:eindex, :] = testout[0].detach().cpu().numpy()
    outindex[sindex:eindex] = data[2]

    print(sindex, eindex)

print(dt.datetime.now())

2024-07-02 10:25:06.484296
step: 1
torch.Size([128, 512])
torch.Size([128, 512])
Step [1/10], Loss: 5.5661
2
torch.Size([128, 512])
0 128
step: 2
torch.Size([128, 512])
torch.Size([128, 512])
Step [2/10], Loss: 5.5783
2
torch.Size([128, 512])
128 256
step: 3
torch.Size([128, 512])
torch.Size([128, 512])
Step [3/10], Loss: 5.5362
2
torch.Size([128, 512])
256 384
step: 4
torch.Size([128, 512])
torch.Size([128, 512])
Step [4/10], Loss: 5.6315
2
torch.Size([128, 512])
384 512
step: 5
torch.Size([128, 512])
torch.Size([128, 512])
Step [5/10], Loss: 5.5928
2
torch.Size([128, 512])
512 640
step: 6
torch.Size([128, 512])
torch.Size([128, 512])
Step [6/10], Loss: 5.5828
2
torch.Size([128, 512])
640 768
step: 7
torch.Size([128, 512])
torch.Size([128, 512])
Step [7/10], Loss: 5.5788
2
torch.Size([128, 512])
768 896
step: 8
torch.Size([128, 512])
torch.Size([128, 512])
Step [8/10], Loss: 5.5422
2
torch.Size([128, 512])
896 1024
step: 9
torch.Size([128, 512])
torch.Size([128, 512])
Step [9/10], Los

In [30]:
outfile = rootdir + 'ResNet_output/RH_input/%s_output.anomaly.daymean.1981-2020.ERA5.mat' % opt_model
print('writing to %s ...' % outfile)
description = 'Just the simCLR encoder output. So in 512 dimension. Use %s model' % opt_model
script = '/global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/step02.ResNet_encoder_production_run.ipynb'
sio.savemat(outfile, {'ResNetoutput':outdata, 'tindex':outindex, 'description':description, 'script':script})

writing to /global/cfs/projectdirs/m1657/liuy351/TallTower/From_XD/ResNet_output/RH_input/R18_output.anomaly.daymean.1981-2020.ERA5.mat ...
